In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import os
import sys
import tarfile
import numpy as np

Using TensorFlow backend.


RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

In [2]:
data_root = os.sep+os.path.join('tmp', 'simpsons')
dest_filename = os.path.join(data_root, 'simpsons_dataset.tar.gz')

filename = '/home/dagutman/dev/KerasSimpsons/simpsons_dataset.tar.gz'

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  if os.path.isdir(root) and not force:
    print('%s already present - Skipping extraction of %s.' % (root, filename))
  else:
    print('Extracting data for %s. This may take a while. Please wait.' % root)
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall(root)
    tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  #print(data_folders)
  return data_folders
#data_folders = maybe_extract(dest_filename)
data_folders = maybe_extract(filename)


/home/dagutman/dev/KerasSimpsons/simpsons_dataset already present - Skipping extraction of /home/dagutman/dev/KerasSimpsons/simpsons_dataset.tar.gz.


In [3]:
img_width, img_height = 64, 64
train_data_dir = os.sep+os.path.join('home', 'dagutman', 'dev', 'KerasSimpsons',  'simpsons_dataset')
validation_data_dir = os.sep+os.path.join('home', 'dagutman', 'dev', 'KerasSimpsons', 'kaggle_simpson_testset')

nb_train_samples = 30000
nb_validation_samples = 990
epochs = 50
batch_size = 32

In [ ]:
## print validation_data_dir
## May need to check the data_folders as I think there are more directories then originally trained on
##data_folders

In [4]:
# Model definition
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


NumLabels = 20
    
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NumLabels))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [5]:
# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255.0,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    fill_mode = 'nearest',
    horizontal_flip=True)

# Only rescaling for validation
valid_datagen = ImageDataGenerator(rescale=1. / 255.0)

In [6]:
# Flows the data directly from the directory structure, resizing where needed
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 19548 images belonging to 20 classes.


In [ ]:
### tunablge parameter ===  batch_size is 1
### other one is image size   64 vs 96  vs 128

In [7]:
validation_generator = valid_datagen.flow_from_directory(
    validation_data_dir + "/",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

simpsonsModel = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)


Found 990 images belonging to 20 classes.
Epoch 1/50
937/937 [==============================] - 148s - loss: 2.8048 - acc: 0.1334 - val_loss: 2.6426 - val_acc: 0.2188
Epoch 2/50
937/937 [==============================] - 149s - loss: 2.3082 - acc: 0.2865 - val_loss: 1.9360 - val_acc: 0.3789
Epoch 3/50
937/937 [==============================] - 150s - loss: 1.8472 - acc: 0.4236 - val_loss: 1.4647 - val_acc: 0.5115
Epoch 4/50
937/937 [==============================] - 150s - loss: 1.5099 - acc: 0.5193 - val_loss: 1.2728 - val_acc: 0.5532
Epoch 5/50
937/937 [==============================] - 150s - loss: 1.3081 - acc: 0.5858 - val_loss: 0.9487 - val_acc: 0.6806
Epoch 6/50
937/937 [==============================] - 150s - loss: 1.1857 - acc: 0.6303 - val_loss: 0.8702 - val_acc: 0.6921
Epoch 7/50
937/937 [==============================] - 150s - loss: 1.0880 - acc: 0.6679 - val_loss: 0.7990 - val_acc: 0.7307
Epoch 8/50
937/937 [==============================] - 149s - loss: 1.0097 - acc: 0.

In [11]:
import h5py
model.save_weights('simpsons_weights.h5')

simpsonsModel.history

{'acc': [0.13335557038621795,
  0.28649586337027722,
  0.42358238824691036,
  0.51934881240312403,
  0.58575717147148954,
  0.63030424343448399,
  0.66794529685662385,
  0.69268748331998931,
  0.70323549030302679,
  0.72524686412617845,
  0.73515677119669232,
  0.74786495862561986,
  0.75326884591317,
  0.76387776888177206,
  0.76838137173226329,
  0.77271514344485781,
  0.78622898317064194,
  0.79026017345691857,
  0.79456898850824531,
  0.80126751165854448,
  0.80367627439258382,
  0.8121080720400794,
  0.82129036562583402,
  0.81891260838969859,
  0.82632772881793182,
  0.83278852569174178,
  0.83553509471859966,
  0.83669112740237372,
  0.84027221779808037,
  0.84507606084867892,
  0.84863242159850438,
  0.84787830264211372,
  0.85083388924360115,
  0.8542834267573004,
  0.85570380255092848,
  0.85878702965551357,
  0.86017344894211956,
  0.85791966904472727,
  0.864076050700467,
  0.85788630905519148,
  0.86460973984245626,
  0.86502535364008726,
  0.86884589728074846,
  0.8694955

In [12]:
## https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/
# Training history
RunStats = """
Found 990 images belonging to 20 classes.
Epoch 1/50
937/937 [==============================] - 148s - loss: 2.8048 - acc: 0.1334 - val_loss: 2.6426 - val_acc: 0.2188
Epoch 2/50
937/937 [==============================] - 149s - loss: 2.3082 - acc: 0.2865 - val_loss: 1.9360 - val_acc: 0.3789
Epoch 3/50
937/937 [==============================] - 150s - loss: 1.8472 - acc: 0.4236 - val_loss: 1.4647 - val_acc: 0.5115
Epoch 4/50
937/937 [==============================] - 150s - loss: 1.5099 - acc: 0.5193 - val_loss: 1.2728 - val_acc: 0.5532
Epoch 5/50
937/937 [==============================] - 150s - loss: 1.3081 - acc: 0.5858 - val_loss: 0.9487 - val_acc: 0.6806
Epoch 6/50
937/937 [==============================] - 150s - loss: 1.1857 - acc: 0.6303 - val_loss: 0.8702 - val_acc: 0.6921
Epoch 7/50
937/937 [==============================] - 150s - loss: 1.0880 - acc: 0.6679 - val_loss: 0.7990 - val_acc: 0.7307
Epoch 8/50
937/937 [==============================] - 149s - loss: 1.0097 - acc: 0.6927 - val_loss: 0.7301 - val_acc: 0.7432
Epoch 9/50
937/937 [==============================] - 149s - loss: 0.9735 - acc: 0.7032 - val_loss: 0.6764 - val_acc: 0.7797
Epoch 10/50
937/937 [==============================] - 152s - loss: 0.9060 - acc: 0.7253 - val_loss: 0.6662 - val_acc: 0.7954
Epoch 11/50
937/937 [==============================] - 147s - loss: 0.8682 - acc: 0.7352 - val_loss: 0.5804 - val_acc: 0.8184
Epoch 12/50
937/937 [==============================] - 149s - loss: 0.8312 - acc: 0.7479 - val_loss: 0.5357 - val_acc: 0.8424
Epoch 13/50
937/937 [==============================] - 150s - loss: 0.8156 - acc: 0.7533 - val_loss: 0.5789 - val_acc: 0.8278
Epoch 14/50
937/937 [==============================] - 151s - loss: 0.7829 - acc: 0.7639 - val_loss: 0.4817 - val_acc: 0.8486
Epoch 15/50
937/937 [==============================] - 151s - loss: 0.7669 - acc: 0.7684 - val_loss: 0.4965 - val_acc: 0.8340
Epoch 16/50
937/937 [==============================] - 148s - loss: 0.7578 - acc: 0.7727 - val_loss: 0.4985 - val_acc: 0.8434
Epoch 17/50
937/937 [==============================] - 150s - loss: 0.7285 - acc: 0.7862 - val_loss: 0.4612 - val_acc: 0.8591
Epoch 18/50
937/937 [==============================] - 148s - loss: 0.7032 - acc: 0.7902 - val_loss: 0.4348 - val_acc: 0.8695
Epoch 19/50
937/937 [==============================] - 149s - loss: 0.6932 - acc: 0.7946 - val_loss: 0.3959 - val_acc: 0.8695
Epoch 20/50
937/937 [==============================] - 148s - loss: 0.6769 - acc: 0.8013 - val_loss: 0.4111 - val_acc: 0.8706
Epoch 21/50
937/937 [==============================] - 152s - loss: 0.6771 - acc: 0.8037 - val_loss: 0.4066 - val_acc: 0.8747
Epoch 22/50
937/937 [==============================] - 150s - loss: 0.6650 - acc: 0.8121 - val_loss: 0.3566 - val_acc: 0.8820
Epoch 23/50
937/937 [==============================] - 147s - loss: 0.6347 - acc: 0.8213 - val_loss: 0.4213 - val_acc: 0.8737
Epoch 24/50
937/937 [==============================] - 148s - loss: 0.6464 - acc: 0.8189 - val_loss: 0.3959 - val_acc: 0.8800
Epoch 25/50
937/937 [==============================] - 148s - loss: 0.6217 - acc: 0.8264 - val_loss: 0.3577 - val_acc: 0.8894
Epoch 26/50
937/937 [==============================] - 149s - loss: 0.6019 - acc: 0.8328 - val_loss: 0.3192 - val_acc: 0.9040
Epoch 27/50
937/937 [==============================] - 147s - loss: 0.5956 - acc: 0.8355 - val_loss: 0.3497 - val_acc: 0.8935
Epoch 28/50
937/937 [==============================] - 147s - loss: 0.5903 - acc: 0.8367 - val_loss: 0.4331 - val_acc: 0.8727
Epoch 29/50
937/937 [==============================] - 150s - loss: 0.5870 - acc: 0.8403 - val_loss: 0.3116 - val_acc: 0.9154
Epoch 30/50
937/937 [==============================] - 148s - loss: 0.5706 - acc: 0.8451 - val_loss: 0.2925 - val_acc: 0.9311
Epoch 31/50
937/937 [==============================] - 148s - loss: 0.5554 - acc: 0.8486 - val_loss: 0.2824 - val_acc: 0.9154
Epoch 32/50
937/937 [==============================] - 148s - loss: 0.5658 - acc: 0.8479 - val_loss: 0.2911 - val_acc: 0.9144
Epoch 33/50
937/937 [==============================] - 148s - loss: 0.5463 - acc: 0.8508 - val_loss: 0.2689 - val_acc: 0.9165
Epoch 34/50
937/937 [==============================] - 148s - loss: 0.5415 - acc: 0.8543 - val_loss: 0.2553 - val_acc: 0.9228
Epoch 35/50
937/937 [==============================] - 148s - loss: 0.5404 - acc: 0.8557 - val_loss: 0.3271 - val_acc: 0.8998
Epoch 36/50
937/937 [==============================] - 148s - loss: 0.5265 - acc: 0.8588 - val_loss: 0.2582 - val_acc: 0.9165
Epoch 37/50
937/937 [==============================] - 149s - loss: 0.5204 - acc: 0.8602 - val_loss: 0.2910 - val_acc: 0.9050
Epoch 38/50
937/937 [==============================] - 148s - loss: 0.5272 - acc: 0.8579 - val_loss: 0.2295 - val_acc: 0.9248
Epoch 39/50
937/937 [==============================] - 148s - loss: 0.5005 - acc: 0.8641 - val_loss: 0.3172 - val_acc: 0.9050
Epoch 40/50
937/937 [==============================] - 148s - loss: 0.5265 - acc: 0.8579 - val_loss: 0.2729 - val_acc: 0.9196
Epoch 41/50
937/937 [==============================] - 148s - loss: 0.5121 - acc: 0.8646 - val_loss: 0.2641 - val_acc: 0.9092
Epoch 42/50
937/937 [==============================] - 149s - loss: 0.5021 - acc: 0.8650 - val_loss: 0.2385 - val_acc: 0.9238
Epoch 43/50
937/937 [==============================] - 151s - loss: 0.4891 - acc: 0.8689 - val_loss: 0.3369 - val_acc: 0.9081
Epoch 44/50
937/937 [==============================] - 157s - loss: 0.4958 - acc: 0.8695 - val_loss: 0.2548 - val_acc: 0.9207
Epoch 45/50
937/937 [==============================] - 157s - loss: 0.4895 - acc: 0.8701 - val_loss: 0.2125 - val_acc: 0.9353
Epoch 46/50
937/937 [==============================] - 157s - loss: 0.4913 - acc: 0.8673 - val_loss: 0.2745 - val_acc: 0.9175
Epoch 47/50
937/937 [==============================] - 158s - loss: 0.4867 - acc: 0.8715 - val_loss: 0.2397 - val_acc: 0.9395
Epoch 48/50
937/937 [==============================] - 149s - loss: 0.4842 - acc: 0.8716 - val_loss: 0.3024 - val_acc: 0.9134
Epoch 49/50
937/937 [==============================] - 148s - loss: 0.4849 - acc: 0.8713 - val_loss: 0.2235 - val_acc: 0.9374
Epoch 50/50
937/937 [==============================] - 146s - loss: 0.4848 - acc: 0.8701 - val_loss: 0.2104 - val_acc: 0.9280"""

In [9]:
from sklearn.metrics import confusion_matrix
confusion_matrix()
# >>> y_true = ["cat", "ant", "cat", "cat", "ant", "bird"]
# >>> y_pred = ["ant", "ant", "cat", "cat", "ant", "cat"]
# >>> confusion_matrix(y_true, y_pred, labels=["ant", "bird", "cat"])
# array([[2, 0, 0],
#        [0, 0, 1],
#        [1, 0, 2]])
# Case-1 I have used like :

# model.compile(loss=’categorical_crossentropy’, optimizer=’Nadam’, metrics=[‘acc’, ‘fmeasure’, ‘precision’, ‘recall’])

# Case-2 and also used :

# def score(yh, pr):
# coords = [np.where(yhh > 0)[0][0] for yhh in yh]
# yh = [yhh[co:] for yhh, co in zip(yh, coords)]
# ypr = [prr[co:] for prr, co in zip(pr, coords)]
# fyh = [c for row in yh for c in row]
# fpr = [c for row in ypr for c in row]
# return fyh, fpr

# pr = model.predict_classes(X_train)
# yh = y_train.argmax(2)
# fyh, fpr = score(yh, pr)
# print ‘Training accuracy:’, accuracy_score(fyh, fpr)
# print ‘Training confusion matrix:’
# print confusion_matrix(fyh, fpr)
# precision_recall_fscore_support(fyh, fpr)

# pr = model.predict_classes(X_test)
# yh = y_test.argmax(2)
# fyh, fpr = score(yh, pr)
# print ‘Testing accuracy:’, accuracy_score(fyh, fpr)
# print ‘Testing confusion matrix:’
# print confusion_matrix(fyh, fpr)
# precision_recall_fscore_support(fyh, fpr)

# What I have observed is that, accuracy of case-1 and case-2 are different?


<function sklearn.metrics.classification.confusion_matrix>

In [12]:
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as img
import pandas as pd
root_dir = '\\tmp\\simpsons\\internet'

# train_generator.class_indices
chardict = {'grampa': 0,
 'apu': 1,
 'bart': 2,
 'burns': 3,
 'chief': 4,
 'comic': 5,
 'edna': 6,
 'homer': 7,
 'brockman': 8,
 'krusty': 9,
 'lenny': 10,
 'lisa': 11,
 'marge': 12,
 'mayor': 13,
 'milhouse': 14,
 'moe': 15,
 'ned': 16,
 'nelson': 17,
 'skinner': 18,
 'sideshow': 19}

rows = 3
cols = 3
fig, ax = plt.subplots(rows, cols, frameon=False, figsize=(7, 7))
fig.suptitle('Internet images', fontsize=20, y = 1.03)
count=0
for i in range(rows):
    for j in range(cols):
        all_files = os.listdir(root_dir)
        imgpath = os.path.join(root_dir, all_files[count])
        img = Image.open(imgpath)
        img = img.resize((img_width, img_height), Image.ANTIALIAS)
        ax[i][j].imshow(img)
        img = img_to_array(img)
        img = img/255.0
        img = img.reshape((1,) + img.shape)
        pred = model.predict(img, batch_size= 1)
        pred = pd.DataFrame(np.transpose(np.round(pred, decimals = 3)))
        pred = pred.nlargest(n = 3, columns = 0)
        pred['char'] = [list(chardict.keys())[list(chardict.values()).index(x)] for x in pred.index]
        charstr = ''
        for k in range(0,3):
            if k < 2:
                charstr = charstr+str(pred.iloc[k,1])+': '+str(pred.iloc[k,0])+'\n'
            else:
                charstr = charstr+str(pred.iloc[k,1])+': '+str(pred.iloc[k,0])
        ec = (0, .8, .1)
        fc = (0, .9, .2)
        count = count + 1
        ax[i][j].text(0, -10, charstr, size=10, rotation=0,
                ha="left", va="top", 
                bbox=dict(boxstyle="round", ec=ec, fc=fc, alpha = 0.7))
plt.setp(ax, xticks=[], yticks=[])
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

ImportError: numpy.core.multiarray failed to import